# Convert UpNote data into dataframe

2024-03-06

In [1]:
# using Pkg
# Pkg.resolve()

In [2]:
using Glob
using DataFrames

In [3]:
using Dates

In [4]:
# Pkg.add("CSV")

In [5]:
using CSV

In [6]:
pwd()

"d:\\develop\\upnote_md_mgr\\notebook"

In [7]:
upnote_path = "../UpNote/General Space"

"../UpNote/General Space"

In [8]:
function getmdfiles(path = upnote_path)
    files = glob("*.md", path)
end    

getmdfiles (generic function with 2 methods)

In [9]:
tfiles = getmdfiles(upnote_path);

In [12]:
length(tfiles)

24673

Data Conversion Function

In [13]:
function convtxt(path)
    s = ""
    update_dt = ""
    create_dt = ""
    cat_mode = false
    isheader = true
    categories = String[]
    bdytxt = String[]
    tags = String[]

    txt = readlines(path)

    for i in 1:length(txt)
        if i > 1
            # skip first line
            s = txt[i]
            if startswith(s,"date:")
                s2 = replace(s, "date: " => "")
                update_dt = DateTime(s2, DateFormat("yyyy-mm-dd HH:MM:SS"))
            elseif startswith(s,"created: ")
                s2 = replace(s, "created: " => "")
                create_dt = DateTime(s2, DateFormat("yyyy-mm-dd HH:MM:SS"))
            elseif startswith(s,"categories:")
                cat_mode = true
                
            elseif s == "---"
                isheader = false
                cat_mode = false
                continue
            else
                if cat_mode
                    s2 = replace(s, r"^- " => "")
                    push!(categories, s2)
                end
            end
            # start of body parser
            if !isheader 
                if startswith(s,r"#+ ")
                    s2 = replace(s, r"#+ " => "")
                    push!(bdytxt,s2)
                elseif contains(s, r"^[\s\*]+$")
                    s2 = replace(s, r"^[\s\*]+$" => "")
                    push!(bdytxt,s2)
                elseif contains(s,r"^#[^[:punct:] ]*$")
                    s2 = replace(s,r"^#" => "")
                    push!(tags, s2)
                else
                    push!(bdytxt,s)
                end
            end # body
        end # i > 1
    end # i
    
    cat_str = join(categories, "|")
    contents = join(bdytxt, "\n")
    tags_str = join(tags, "|")
    
    return(Dict("fpath" => basename(path), "update" => update_dt, "created" => create_dt, "category" => cat_str, "tags" => tags_str, "contents" => contents))
end

convtxt (generic function with 1 method)

## trial

In [10]:
f1 = tfiles[1]

"../UpNote/General Space\\- 名刺.md"

In [19]:
txt = readlines(f1);

more data

In [14]:
fl1 = tfiles[1:10]

10-element Vector{String}:
 "../UpNote/General Space\\- 名刺.md"
 "../UpNote/General Space\\0030.md"
 "../UpNote/General Space\\0042.md"
 "../UpNote/General Space\\0音孔.md"
 "../UpNote/General Space\\1,2番管.md"
 "../UpNote/General Space\\1,3,4-2 抜差組立 中管ピン径変更.md"
 "../UpNote/General Space\\1-3レバー 2.md"
 "../UpNote/General Space\\1-3レバー.md"
 "../UpNote/General Space\\1.5 スタイラス.md"
 "../UpNote/General Space\\1.5スタイラス.md"

In [15]:
xx = DataFrame()

for f in fl1
    append!(xx, convtxt(f))
end

In [16]:
xx

Row,category,contents,created,fpath,tags,update
,String,String,DateTime,String,String,DateTime
1,名刺,\n\\- 名刺\n\n池田 信次郎日新\n\n\n\nDSI成形機の輸送\n\n![](Files/Evernote%20Snapshot%2020170121%20200204.png),2017-01-21T20:01:05,- 名刺.md,,2017-01-21T20:01:34
2,仕事_Evernote,\n0030\n\n1番管正寸切治具のベースへの取り付け孔をφ11にするようセガワにメール連絡した。\n\n図面も訂正しておいた。\n\n\n\nその他、保全班に追加工依頼した以下のものの図面を訂正。\n\n\n\n\n,2014-06-05T11:06:02,0030.md,HR869,2014-06-05T11:06:15
3,仕事_Evernote,\n0042\n\n枝管3Aの寸切治具、浅羽ではなくYMC用だった。\n\nで、ボルト孔が小さいので、保全班の孔広げ待ちに追加した。\n\n図面訂正した。\n\n\n\n\n,2014-06-05T11:06:15,0042.md,HR869,2014-06-11T16:06:42
4,仕事_Evernote,\n0音孔\n\nボディ測定結果を確認。\n\n大野さんにメール。\n\n\n\n結果、+0/-0.5の公差で振り直してくれる。\n\n\n\nPLズレは、テール部のみ0.05許容。\n\n\n\n表の音孔は+/- 0.1。\n\n\n\n\n\n\n\n\n,2017-02-09T15:02:23,0音孔.md,分岐管,2017-02-09T18:02:36
5,仕事_Evernote,"\n1,2番管\n\nEMCによるナミ修正品の確認を徳富さんと見た。\n\nまだダクってるのがよく見えるので、やはりヤスリとかで修正しないとだめだよな。、ってことでサテライトに回収。\n\n1番管は増試品なんだが、もう一度山隆で修正?松隈さんに相談する。\n\n\n\n2番管はベーダーによる修正で結構アラビ消せてる。が、こちらはシームローラー変更がメインの対策になるので、バックアッププラン。\n\n\n\n清田さんと会話。\n\n2番管はCBとBBで、バルジ3回目からしか変わらないのでそれらが電算でまとめられて欲しいな。→曲げ上がりまではCB共通で構成を組むように伝えたので、生産からは44班の中で適宜コントロールできるようになるはず。\n\n\n\n写真、三宅氏が撮っていた。\n\n\n",2022-07-25T16:07:58,"1,2番管.md",CBBB623,2022-07-25T23:07:08
6,仕事_Evernote,"\n1,3,4-2 抜差組立 中管ピン径変更\n\n中管の嵌合径が大きなクリアランスを取っていることに気づいてなかった佐山さんの設計ミスを修正する図面製図。\n\nたまたま全部の部品が共通だったので、修正は比較的楽だった。\n\n図面が旧来の1ファイルに書いてあるタイプだったので、部品ごとに分割してから修正を入れた。\n\n\n\n休日深夜枠。\n\n\n\n\n\n\n",2022-02-20T02:02:34,"1,3,4-2 抜差組立 中管ピン径変更.md",CBBB623,2022-02-20T02:02:42
7,仕事_Evernote,\n1-3レバー\n\n原さんから、869 1-3レバー変更の依頼がPJから来たら対応してほしいとのこと。\n\n\n\n3レバーのブランク長さは要確認。\n\n原さんの記憶では、一度曲げの長さが合わなかった。\n\n\n,2016-05-25T14:05:33,1-3レバー 2.md,HR869,2016-05-25T14:05:11
8,仕事_Evernote,\n1-3レバー\n\n室園氏。\n\n\n\n設変対応。\n\n何をやればいいのかを室園氏に口頭で説明した。\n\n1. 抜き型の中身と曲げ型のガイドを設計して手配。\n2. 和田精機と中身の組み替えと在庫（2ヶ月分ぐらい？）の製作と試作で打合せ。\n3. 交換後の試作を山本のロー付けバフ上がりまで。\n4. 設計で評価\n5. 生産切り替えのアクション\n6. 余った過去部品は買い取り？\n\n\n\n型設計する予定の吉田氏には、室園氏から説明してもらう。\n\n\n\n\n\n\n\n\n,2016-07-08T13:07:01,1-3レバー.md,HR869,2016-07-08T14:07:00
9,仕事_Evernote,\n1.5 スタイラス\n\n渡邊氏から電話。新しい2本が納品されたが、棚に入れようと思ったら予備があったよ？\n\n→うーん、前回探した時に何故見つけられなかった？\n\n\n\n\n\n\n,2016-02-09T14:02:10,1.5 スタイラス.md,3次元,2016-02-09T14:02:11


In [17]:
CSV.write("test.csv",xx)

"test.csv"

In [14]:
d1 = DataFrame(fpath = "test", contents = "contents\ncontents")
d2 = DataFrame(fpath = "test2", contents = "next item")

Row,fpath,contents
,String,String
1,test2,next item


In [12]:
append!(d1,d2)

Row,fpath,contents
,String,String
1,test,contents\ncontents
2,test2,next item


In [15]:
DataFrame("fpath" => "test", "contents" => "contents\ncontents")

Row,fpath,contents
,String,String
1,test,contents\ncontents


In [16]:
basename(pwd())

"notebook"

In [19]:
for (i,v) in enumerate(["a","b"])
    println(i,v)
end

1a
2b
